<a href="https://colab.research.google.com/github/Fawzy-AI-Explorer/X-From-Scratch/blob/main/Linear_Regression-From_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Linear Regression From Scratch
Table of content:
- [Goals](#goals)
- [Tools](#tools)
- [Prediction && Accuracy](#predection_accuracy)
- [Compute Cost With Multiple Variables](#cost)
- [Gradient Descent With Multiple Variables](#gradient_descent)
- [Final Implementation](#final_implementation)
- [LICENSE](#license)

## <a name="goals">Goals<a>
In this toturial, we will:

- Implement the linear regression model
- Implement the gradient descent algorithm to train the model
- Implement $R^2 Score$ to calculate the accuracy<br><br>

*All from scratch*

## <a name="tools">Tools<a>
In this toturial, we will make use of:
- math, This module provides access to the mathematical functions defined by the C standard
- pandas, a Python library used for working with data sets
- NumPy, a popular library for scientific computing
- seaborn, a Python data visualization library based on matplotlib
- Matplotlib, a popular library for plotting data

In [7]:
import math, copy
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

## <a name="predection_accuracy">Prediction && Accuracy<a>
We will build linear regression model so the prediction will be:<br>
$$ f_{\mathbf{w},b}(\mathbf{x}^{(i)}) = \mathbf{w} \cdot \mathbf{x}^{(i)} + b  \tag{1} $$
<br><br>
The most widely used formula to calculate the accuracy is r2_score so we will use it:
$$R^2=1-\frac{RSS}{TSS}\tag{2}$$<br>
Where:
$$RSS=\sum\limits_{i = 0}^{m-1} (y^{(i)} - f_{\mathbf{w},b}(\mathbf{x}^{(i)}))^2 \tag{3}$$<br>
$$TSS=\sum\limits_{i = 0}^{m - 1} (y^{(i)} - y̅)^2\tag{4}$$<br>
$$y̅=\frac{1}{m}\sum\limits_{i=0}^{m-1} (y^{(i)})\tag{5}$$

In [8]:
def predict(X, w, b):
    """
    single predict using linear regression
    Args:
      X (ndarray): Shape (m, n) example with multiple features
      w (ndarray): Shape (n,) model parameters
      b (scalar):             model parameter

    Returns:
      p (scalar):  prediction
     """

    p = np.dot(X, w) + b
    return p


In [9]:
def score(X, y, w, b):
    """
    Returns the accuracy of the model
    Args:
    X (ndarray): Shape(m, n) examples with multiple features
    y (ndarray): Shape (m,) the actual target values
    w (ndarray): Shape (n,) model parameters
    b (scalar) : model parameter

    Returns:
      accuracy (scalar): the accuracy of the model
    """
    m, n = X.shape
    RSS = 0.
    TSS = 0.
    y_mu = np.mean(y)

    for i in range(m):
        RSS += (y[i] - predict(X[i], w, b)) ** 2
        TSS += (y[i] - y_mu) ** 2

    score = 1 - RSS / TSS
    print("The accuracy of our model is {}%".format(round(score, 2) * 100))


## <a name="cost">Compute Cost With Multiple Variables<a>
The equation for the cost function with multiple variables $J(\mathbf{w},b)$ is:
$$J(\mathbf{w},b) = \frac{1}{2m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)})^2 \tag{1}$$


In [10]:
def compute_cost(X, y, w, b):
    """
    Computes the cost function for linear regression.

    Args:
      X (ndarray (m,n)): Data, m examples and n features
      y (ndarray (m,)): target values
      w (ndarray (m,)): model parameters
      b (scalar)    : model parameter

    Returns
        total_cost (float): The cost of using w,b as the parameters for linear regression
               to fit the data points in x and y
    """

    m = X.shape[0]
    cost = 0.0
    for i in range(m):
      f_wb_i = predict(X[i], w, b);
      cost += (f_wb_i - y[i]) ** 2

    cost /= (2 * m)

    return cost

## <a name="gradient_descent">Gradient Descent With Multiple Variables<a>
Gradient descent for multiple variables:

$$\begin{align*} \text{repeat}&\text{ until convergence:} \; \lbrace \newline\;
& w_j = w_j -  \alpha \frac{\partial J(\mathbf{w},b)}{\partial w_j} \tag{1}  \; & \text{for j = 0..n-1}\newline
&b\ \ = b -  \alpha \frac{\partial J(\mathbf{w},b)}{\partial b}  \newline \rbrace
\end{align*}$$

where, n is the number of features, parameters $w_j$,  $b$, are updated simultaneously and where  

$$
\begin{align}
\frac{\partial J(\mathbf{w},b)}{\partial w_j}  &= \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)})x_{j}^{(i)} \tag{2}  \\
\frac{\partial J(\mathbf{w},b)}{\partial b}  &= \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)}) \tag{3}
\end{align}
$$
* m is the number of training examples in the data set

    
*  $f_{\mathbf{w},b}(\mathbf{x}^{(i)})$ is the model's prediction, while $y^{(i)}$ is the target value


In [11]:
def compute_gradient(X, y, w, b):
    """
    Computes the gradient for linear regression
    Args:
      X (ndarray (m,n)): Data, m examples with n features
      y (ndarray (m,)) : target values
      w (ndarray (n,)) : model parameters
      b (scalar)       : model parameter

    Returns:
      dj_dw (ndarray (n,)): The gradient of the cost w.r.t. the parameters w.
      dj_db (scalar):       The gradient of the cost w.r.t. the parameter b.
    """

    m, n = X.shape
    dj_dw = np.zeros((n,))
    dj_db = 0.

    for i in range(m):
      f_wb_i = predict(X[i], w, b)
      err = (f_wb_i - y[i])
      dj_dw = dj_dw + (err * X[i])
      dj_db += err

    dj_dw /= m
    dj_db /= m

    return dj_dw, dj_db


In [12]:
def gradient_descent(X, y, w_in, b_in, cost_function, gradient_function, alpha, num_iters):
    """
    Performs batch gradient descent to learn w and b. Updates w and b by taking
    num_iters gradient steps with learning rate alpha

    Args:
      X (ndarray (m,n))   : Data, m examples with n features
      y (ndarray (m,))    : target values
      w_in (ndarray (n,)) : initial model parameters
      b_in (scalar)       : initial model parameter
      cost_function       : function to compute cost
      gradient_function   : function to compute the gradient
      alpha (float)       : Learning rate
      num_iters (int)     : number of iterations to run gradient descent

    Returns:
      w (ndarray (n,)) : Updated values of parameters
      b (scalar)       : Updated value of parameter
      """

    # An array to store cost J and w's at each iteration primarily for graphing later
    J_history = []
    W_history = []
    b_history = []
    w = copy.deepcopy(w_in)
    b = b_in
    n = 1
    if (len(X.shape) > 1):
      n = X.shape[1]

    for i in range(num_iters):
      dj_dw, dj_db = gradient_function(X, y, w, b)
      w = w - (alpha * dj_dw)
      b -= alpha * dj_db

      if i < 100000:
        J_history.append(cost_function(X, y, w, b))
        W_history.append(copy.deepcopy(w))
        b_history.append(copy.deepcopy(b))

        # Print cost every at intervals 10 times or as many iterations if < 10
        if i % math.ceil(num_iters / 10) == 0:
          print("{:>8}  cost={:>1.5e}".format(i, cost_function(X, y)), end='')
          for i in range(n):
            print("  w[{}]={:>1.1e}".format(i, w[i]), end='')
          print("  b={:>1.1e}".format(b), end='')
          for i in range(n):
            print("  dj_dw[{}]={:>1.1e}".format(i, dj_dw[i]), end='')
          print("  dj_db={:>1.1e}".format(dj_db))

    return w, b, J_history, W_history, b_history


## <a name="final_implementation">Final Implementation<a>
Now we will grap all this together to make the complete model that can be used easily later.<br><br>
*NOTE*: we will rename some methods just to keep up with the original model

In [13]:
"""Define a linear regressino class."""


class LinearRegression():
    """Representation of a linear regression model.
    the model predicts a number form  infinitely many possible outputs.
    """

    def __init__(self, learning_rate=1e-4, max_iter=1000):
      """Initialize the linear regression model

        Args:
          learning_rage (float) : The number indicates the step in gradient descent.
          max_iter (int) : The maximum number of passes over the training data.
      """

      self.learning_rate = learning_rate
      self.max_iter = max_iter
      self.w, self.b = None, 0.

    def fit(self, X, y, print_history=True):
      """
      Performs batch gradient descent to learn w and b. Updates w and b by taking
      num_iters gradient steps with learning rate alpha

      Args:
        X (ndarray (m,n))   : Data, m examples with n features
        y (ndarray (m,))    : target values

      Returns:
        self (object) : Fitted model estimator.
        """

      # An array to store cost J and w's at each iteration primarily for graphing later
      J_history = []
      W_history = []
      b_history = []
      n = 1
      if (len(X.shape) > 1):
        n = X.shape[1]
      self.w = np.zeros((n, ))
      self.b = 0.

      for i in range(self.max_iter):
        dj_dw, dj_db = self.compute_gradient(X, y)
        self.w = self.w - (self.learning_rate * dj_dw)
        self.b -= self.learning_rate * dj_db

        if i < 100000:
          J_history.append(self.compute_cost(X, y))
          W_history.append(copy.deepcopy(self.w))
          b_history.append(copy.deepcopy(self.b))

        # Print cost every at intervals 10 times or as many iterations if < 10
        if i % math.ceil(self.max_iter / 10) == 0 and print_history:
          print("{:>8}  cost={:>1.5e}".format(i, self.compute_cost(X, y)), end='')
          for i in range(n):
            print("  w[{}]={:>1.1e}".format(i, self.w[i]), end='')
          print("  b={:>1.1e}".format(self.b), end='')
          for i in range(n):
            print("  dj_dw[{}]={:>1.1e}".format(i, dj_dw[i]), end='')
          print("  dj_db={:>1.1e}".format(dj_db))

      return self


    def compute_gradient(self, X, y):
      """
      Computes the gradient for linear regression
      Args:
        X (ndarray (m,n)): Data, m examples with n features
        y (ndarray (m,)) : target values

      Returns:
        dj_dw (ndarray (n,)): The gradient of the cost w.r.t. the parameters w.
        dj_db (scalar):       The gradient of the cost w.r.t. the parameter b.
      """

      m = X.shape[0]
      n = 1
      if (len(X.shape) > 1):
        n = X.shape[1]
      dj_dw = np.zeros((n,))
      dj_db = 0.

      for i in range(m):
        f_wb_i = self.predict(X[i])
        err = (f_wb_i - y[i])
        dj_dw = dj_dw + (err * X[i])
        dj_db += err

      dj_dw /= m
      dj_db /= m

      return dj_dw, dj_db


    def compute_cost(self, X, y):
      """
      Computes the cost function for linear regression.

      Args:
        X (ndarray (m,n)): Data, m examples and n features
        y (ndarray (m,)): target values

      Returns
          total_cost (float): The cost of using w,b as the parameters for linear regression
                to fit the data points in x and y
      """

      m = X.shape[0]
      cost = 0.0
      for i in range(m):
        f_wb_i = self.predict(X[i])
        cost += (f_wb_i - y[i]) ** 2

      cost /= (2 * m)

      return cost

    def predict(self, X):
      """
      single predict using linear regression
      Args:
        X (ndarray): Shape (m, n) example with multiple features

      Returns:
        p (scalar):  prediction
      """

      p = np.dot(X, self.w) + self.b
      return p

    def score(self, X, y):
      """
      Returns the accuracy of the model
      Args:
      X (ndarray): Shape(m, n) examples with multiple features
      y (ndarray): Shape (m,) the actual target values

      Returns:
        accuracy (scalar): the accuracy of the model
      """
      m = X.shape[0]
      n = 1
      if (len(X.shape) > 1):
        n = X.shape[1]
      RSS = 0.
      TSS = 0.
      y_mu = np.mean(y)

      for i in range(m):
          RSS += (y[i] - self.predict(X[i])) ** 2
          TSS += (y[i] - y_mu) ** 2

      score = 1 - RSS / TSS
      return round(score, 2)

    def get_params(self):
      """
      Get parameters for this estimator.

      Returns:
        params (dict) : Parameter names mapped to their values.
      """
      params = {}
      params["learning_rate"] = self.learning_rate
      params["max_iter"] = self.max_iter
      if self.w is not None:
        for i in range(len(self.w)):
            p_name = "w" + str(i)
            params[p_name] = self.w[i]
      if self.b is not None:
        params["b"] = self.b

      return params

    def set_params(self, **params):
      """
      Set the parameters of this estimator.

      Args:
        Estimator parameters.

      Returns:
        self (estimator instance) : Estimator instance.
      """


## <a name="test">Test the model<a>
Now let's test the model on the <a href="https://www.kaggle.com/datasets/kolawale/focusing-on-mobile-app-or-website">ecommerce data set<a>

In [15]:
# Load the data set
url = 'https://raw.githubusercontent.com/Ad7amstein/Linear_Regression-E-commerce/main/Ecommerce%20Customers.csv'
data = pd.read_csv(url)
# extract the four main features
X = data.values[:, 3:7]
y = data.values[:, 7]
# split the data to train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
# make instance form the model
lm = LinearRegression(max_iter=100000, learning_rate=1e-4)
# run the model on the training data
lm.fit(X_train, y_train, print_history=True)
# print the model parameters
lm.get_params()
# print the score
print("The score of our model is {}%".format(lm.score(X_test, y_test) * 100))

       0  cost=7.18746e+04  w[0]=1.7e+00  w[1]=6.1e-01  w[2]=1.9e+00  w[3]=1.9e-01  b=5.0e-02  dj_dw[0]=-1.7e+04  dj_dw[1]=-6.1e+03  dj_dw[2]=-1.9e+04  dj_dw[3]=-1.9e+03  dj_db=-5.0e+02
   10000  cost=5.83830e+02  w[0]=1.0e+01  w[1]=2.3e+01  w[2]=-6.9e+00  w[3]=4.0e+01  b=-2.5e-01  dj_dw[0]=-2.0e+00  dj_dw[1]=-1.2e+01  dj_dw[2]=7.7e+00  dj_dw[3]=-2.2e+01  dj_db=4.3e-01
   20000  cost=3.04426e+02  w[0]=1.1e+01  w[1]=3.1e+01  w[2]=-1.2e+01  w[3]=5.4e+01  b=-6.7e-01  dj_dw[0]=-6.2e-01  dj_dw[1]=-4.2e+00  dj_dw[2]=2.6e+00  dj_dw[3]=-7.3e+00  dj_db=4.2e-01
   30000  cost=2.72291e+02  w[0]=1.2e+01  w[1]=3.3e+01  w[2]=-1.3e+01  w[3]=5.8e+01  b=-1.1e+00  dj_dw[0]=-1.9e-01  dj_dw[1]=-1.5e+00  dj_dw[2]=8.8e-01  dj_dw[3]=-2.4e+00  dj_db=4.2e-01
   40000  cost=2.68449e+02  w[0]=1.2e+01  w[1]=3.4e+01  w[2]=-1.4e+01  w[3]=6.0e+01  b=-1.5e+00  dj_dw[0]=-6.3e-02  dj_dw[1]=-5.3e-01  dj_dw[2]=2.9e-01  dj_dw[3]=-8.1e-01  dj_db=4.2e-01
   50000  cost=2.67851e+02  w[0]=1.2e+01  w[1]=3.4e+01  w[2]=-1.4e+01 

## <a name="license">Jupyter Notebook License<a>
### Author: Adham Allam
### How to reach me ?
- <a href="https://www.linkedin.com/in/adham-allam-284486254/">Linkedin<a>
- <a href="https://linktr.ee/Adham.3llam">Linktree<a>

### Terms:
1. Free to use for learning purposes.
2. Attribution to the author, Adham Allam, is required.
3. Permission to edit and explore, but derivative works must reference the original notebook and author.
4. No warranties provided.<br>

By using this notebook, you agree to these terms.